In [1]:
import csv
urls_uueduudised = []

with open('uueduudised_rubriigid_arhiiv.csv', 'r', encoding = 'utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        if line != []:
            urls_uueduudised.append(line[0])

In [43]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

pealkirjad = []
tekstid = []
väljaanded = []
ajad =[]
reklaamiteade = 'Järgneb Google reklaam. Uute uudiste toimetus ei vastuta Google algoritmide poolt personaalselt teile suunatud reklaamide sisu eest. Soovitame usaldada ainult tuntud ja usaldusväärseid kaubamärke.'
for url in tqdm(urls_uueduudised[:7000]):
    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    pealkiri = soup.find("meta", {"property" : "og:title"})['content']
    pealkiri = pealkiri.replace('\xa0','')
    väljaanne = soup.find("meta", {"property" : "og:site_name"})['content']
    aeg = soup.find("meta", {"itemprop" : "datePublished"})['content']

    artikkel = soup.findAll("div", {"itemprop" : "articleBody"})
    #artikkel = results.find_all("div")
    yhendatud_tekst = ''
    for art in artikkel:
        loik = art.find_all("p")
        for l in loik:
            if reklaamiteade not in l.text:
                text = l.text.replace('\n','')
                text = text.replace('\xa0','')
                text = text + ' '
                yhendatud_tekst += text
    tekstid.append(yhendatud_tekst)
       
    pealkirjad.append(pealkiri)
    väljaanded.append(väljaanne)
    ajad.append(aeg)

100%|████████████████████████████████████████████████████████████████████████████| 7000/7000 [2:48:12<00:00,  1.44s/it]


In [44]:
artiklid_7000 = {'title': pealkirjad, 'text': tekstid,'subject': väljaanded, 'date': ajad, 'link': urls_uueduudised[:7000]}

In [46]:
data_uueduudised_rubriigid = pd.DataFrame(artiklid_7000, columns = ['title', 'text','subject', 'date', 'link'])

In [47]:
data_uueduudised_rubriigid.shape

(7000, 5)

In [49]:
data_uueduudised_rubriigid.to_csv('data_uueduudised_rubriigid_arhiiv.csv',index=False)